In [261]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.graphics.tsaplots import plot_acf

# Read in the data
df = pd.read_csv('Datasets/cleaned_data.csv')

df['date'] = pd.to_datetime(df['date'])
#df.loc[df['id'] == 'AS14.01']


In [262]:
final_df = pd.DataFrame()
distinct_keys = df['id'].unique()
period_time = 5

agg_dict = {'mood': 'last', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean',
            'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum',
            'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum',
            'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum',
            'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum',
            'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

for key in distinct_keys:

    distinct_df = df[df['id'] == key]

    min_date = distinct_df['date'].min()
    max_date = distinct_df['date'].max()

    # Create a list of dates for which you want to loop
    dates_to_loop = pd.date_range(start=min_date, end=max_date, freq='D')

    # Loop through the dates and filter the dataframe for each date and the next 5 days
    new_df = pd.DataFrame()
    period = 0
    for date in dates_to_loop:
        filtered_df = distinct_df.loc[(df['date'] >= date) & (df['date'] <= date + pd.DateOffset(days=period_time))]

        period += 1
        filtered_df['period'] = period
        filtered_df = filtered_df.groupby('period').agg(agg_dict).reset_index()  

        new_df = new_df.append(filtered_df)

    new_df['id'] = key
    total_rows = len(new_df)

    # Remove last rows period_time rows from the dataframe, since those are the same
    new_df = new_df.iloc[:total_rows - period_time]

    final_df = final_df.append(new_df)

final_df


C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,appCat.communication,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,id
0,1,6.60000,0.350000,0.416667,0.132052,83841.407001,19.0,2.0,8224.114,45017.260,...,243.514,0.000,175.216,753.592,15584.127,1372.555,235.223,30.386,1181.451,AS14.01
0,2,7.00000,0.350000,0.516667,0.136296,83304.650000,15.0,3.0,6314.223,45672.882,...,236.189,0.000,185.461,572.373,12011.008,504.424,235.223,30.386,761.726,AS14.01
0,3,6.40000,0.150000,0.550000,0.112766,82085.978000,17.0,2.0,15645.389,41576.797,...,304.279,233.036,185.461,700.181,12084.117,1600.128,235.223,30.386,945.822,AS14.01
0,4,8.00000,0.150000,0.583333,0.100342,83634.767999,21.0,3.0,16311.773,39060.170,...,260.876,233.036,185.461,796.952,11656.166,1652.563,235.223,0.000,1516.373,AS14.01
0,5,7.50000,-0.066667,0.708333,0.095364,73110.630999,11.0,3.0,15859.182,31088.046,...,226.770,233.036,182.451,804.397,8600.225,1232.758,301.700,0.000,1375.937,AS14.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,48,7.00000,0.191667,0.869444,0.098189,25301.214997,19.0,10.0,7636.286,14762.938,...,0.000,0.000,0.000,234.954,1871.304,225.737,0.000,0.000,45.854,AS14.24
0,49,7.50000,0.108333,0.744444,0.094348,26329.468997,35.0,10.0,7519.069,15849.366,...,0.000,0.000,0.000,264.349,1871.304,225.737,0.000,0.000,47.926,AS14.24
0,50,8.00000,0.108333,0.633333,0.100104,26846.925997,30.0,12.0,10953.277,14276.017,...,0.000,0.000,0.000,189.494,1218.186,225.737,0.000,0.000,47.926,AS14.24
0,51,6.50000,-0.183333,0.675000,0.106847,23509.534997,29.0,12.0,10357.920,12177.546,...,0.000,0.000,0.000,185.436,613.077,225.737,0.000,0.000,14.753,AS14.24


In [263]:
distinct_df = final_df[final_df['id'] == 'AS14.01']
distinct_df

,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,appCat.communication,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,id
0,1,6.600000,0.350000,0.416667,0.132052,83841.407001,19.0,2.0,8224.114,45017.260,...,243.514,0.000,175.216,753.592,15584.127,1372.555,235.223,30.386,1181.451,AS14.01
0,2,7.000000,0.350000,0.516667,0.136296,83304.650000,15.0,3.0,6314.223,45672.882,...,236.189,0.000,185.461,572.373,12011.008,504.424,235.223,30.386,761.726,AS14.01
0,3,6.400000,0.150000,0.550000,0.112766,82085.978000,17.0,2.0,15645.389,41576.797,...,304.279,233.036,185.461,700.181,12084.117,1600.128,235.223,30.386,945.822,AS14.01
0,4,8.000000,0.150000,0.583333,0.100342,83634.767999,21.0,3.0,16311.773,39060.170,...,260.876,233.036,185.461,796.952,11656.166,1652.563,235.223,0.000,1516.373,AS14.01
0,5,7.500000,-0.066667,0.708333,0.095364,73110.630999,11.0,3.0,15859.182,31088.046,...,226.770,233.036,182.451,804.397,8600.225,1232.758,301.700,0.000,1375.937,AS14.01
0,6,7.400000,-0.150000,0.725000,0.090720,63472.110000,17.0,2.0,17787.819,26143.932,...,224.971,284.212,182.451,894.132,8031.409,1416.139,95.767,0.000,1363.349,AS14.01
0,7,6.000000,-0.083333,0.625000,0.093576,64222.215002,18.0,2.0,18005.839,23250.705,...,720.466,518.953,274.839,917.824,8479.417,3030.760,95.767,0.000,1847.620,AS14.01
0,8,6.500000,-0.075000,0.575000,0.083977,68138.576003,20.0,1.0,20390.861,23234.349,...,926.024,518.953,182.869,1400.833,10972.828,2983.446,95.767,0.000,2009.962,AS14.01
0,9,6.400000,-0.175000,0.475000,0.076233,84616.922004,19.0,1.0,12649.730,36805.598,...,913.092,285.917,182.869,1388.784,13614.328,1869.519,95.767,0.000,1891.017,AS14.01
0,10,6.200000,-0.175000,0.375000,0.085981,88673.711004,17.0,1.0,12654.929,39566.534,...,913.092,285.917,182.869,1395.991,15227.637,1975.325,95.767,0.000,1419.072,AS14.01


In [61]:
# Still need to remove the first column
df.to_csv('Datasets/cleaned_data_feature_egineering.csv', index=False)